In [ ]:
import sys
sys.path.append('../..')
%load_ext autoreload
%autoreload 2

In [ ]:
import spatialpy
import numpy

In [ ]:
class Simple_Diffusion(spatialpy.Model):
    """ Initial condition is a delta function at and off-center point. 
        The solution should be a Gaussian, and periodic boundary conditions
        should allow it to wrap. """

    def __init__(self, model_name="simple_diffusion"):
        spatialpy.Model.__init__(self,model_name)
        
        # System constants
        D = 0.01

        # Define Species
        A = spatialpy.Species(name="A",diffusion_constant=D)
        self.add_species([A])

        # A unit square
        self.mesh = spatialpy.Mesh.generate_unit_square_mesh(40,40, periodic=True)

        # Define Subdomains
        # Subdomains are NOT complete!
        # Please complete these and update this model!
        self.add_subdomain(, 1)

        # Place the A molecules in the voxel nearest the center of the square
        self.add_initial_condition(
            spatialpy.PlaceInitialCondition(A,100000,[0,0,0])
        )

        self.timespan(numpy.linspace(0,5,1000))

In [ ]:
model = Simple_Diffusion()

In [ ]:
%time result = model.run()

In [ ]:
#result.display('A',-1)

In [ ]:
def display(result, species, timepoint, opacity=1.0, wireframe=True, width=500):
    data = result.get_species(species,timepoint,concentration=True)
    fun = pyurdme.DolfinFunctionWrapper(result.model.mesh.get_function_space())
    vec = fun.vector()
    nd = data.shape[0]
    if nd == len(vec):
        for i in range(nd):
            vec[i]=data[i]
    else:
        v2d= result.get_v2d()
        for i in range(len(vec)):
            vec[i]=data[i]
    fun.display(opacity=opacity, wireframe=wireframe, width=width)

In [ ]:
display(result, 'A', 200)

In [ ]:
model.mesh